In [46]:
import requests
from bs4 import BeautifulSoup

In [47]:
def get_links_on_page(page):
    r = requests.get('https://www.the-numbers.com/movie/budgets/all' + page)
    soup = BeautifulSoup(r.text, 'html.parser')
    all_links = soup.find_all('a')
    valid_links = []
    counter = 0
    init_link = 'https://www.the-numbers.com'
    for link in all_links:
        if(link.get('href') and not  link.get('href').startswith('/movie/budgets')\
            and link.get('href').startswith('/movie/') and counter < 100):
            #print(link.get('href'))
            valid_links.append(init_link + link.get('href'))
            counter+=1

    return valid_links

In [48]:
def get_movie_info(soup_movie):
    current_movie = {'Name' : '',
                    'Year' : '',
                    'Languages' : '',
                    'Countries': '',
                    'Genres': '',
                    'Budget': '',
                    'runtime': '',
                    'rating': '',
                    'domestic_gross': '',
                    'foreign_gross': '',
                    'worldwide_gross': '',
                    }
    movie_title_with_year = soup_movie.find('h1').text
    year = movie_title_with_year[movie_title_with_year.find("(")+1:movie_title_with_year.find(")")]
    movie_title = movie_title_with_year[:movie_title_with_year.find("(")]

    current_movie['Name'] = movie_title
    current_movie['Year'] = year

    all_td = soup_movie.find_all('td')
    for i in range(len(all_td)):
        if all_td[i].find('b'): 
            if all_td[i].find('b').text=='Production Countries:':
                current_movie['Countries'] = all_td[i+1].text
            elif all_td[i].find('b').text=='Domestic Box Office':
                current_movie['domestic_gross'] = all_td[i+1].text
            elif all_td[i].find('b').text=='International Box Office':
                current_movie['foreign_gross'] = all_td[i+1].text
            elif all_td[i].find('b').text=='Worldwide Box Office':
                current_movie['worldwide_gross'] = all_td[i+1].text
            elif all_td[i].find('b').text=='Running Time:':
                current_movie['runtime'] = all_td[i+1].text
            elif all_td[i].find('b').text=='Genre:':
                current_movie['Genres'] = all_td[i+1].text
            elif all_td[i].find('b').text=='Languages:':
                current_movie['Languages'] = all_td[i+1].text
            elif all_td[i].find('b').text=='MPAA Rating:':
                current_movie['rating'] = all_td[i+1].text
            elif all_td[i].find('b').text=='Production Budget:':
                current_movie['Budget'] = all_td[i+1].text
            
    return current_movie


In [44]:
pages_to_scrape = 10
links_table = []
for i in range(pages_to_scrape):
    if i == 0:
        page=''
    else:
        page = '/' + str(i*100 + 1)
    links = get_links_on_page(page)
    print(links[0])
    links_table.append(links)
print(len(links_table))
    

https://www.the-numbers.com/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=summary
https://www.the-numbers.com/movie/Wonder-Woman-1984-(2020)#tab=summary
https://www.the-numbers.com/movie/Sorcerers-Apprentice-The#tab=summary
https://www.the-numbers.com/movie/Revenant-The-(2015)#tab=summary
https://www.the-numbers.com/movie/Ferdinand#tab=summary
https://www.the-numbers.com/movie/Treasure-Planet#tab=summary
https://www.the-numbers.com/movie/Promise-The-(2017)#tab=summary
https://www.the-numbers.com/movie/Space-Jam#tab=summary
https://www.the-numbers.com/movie/Munich-(2005)#tab=summary
https://www.the-numbers.com/movie/Ruby-Gillman-Teenage-Kraken-(2023)#tab=summary
10


In [51]:
import pandas as pd
from tqdm import tqdm

counter = 0
pages_to_scrape = 10
start = 0

for i in range(start, start+pages_to_scrape):
    if i == 0:
        page=''
        filename = "/0"
    else:
        page = '/' + str(i*100 + 1)
        filename = page
    links = get_links_on_page(page)
    print('got all links')
    all_movies_info = []
    for k in tqdm(range(len(links))):
        counter+=1
        r_movie = requests.get(links[k])
        soup_movie = BeautifulSoup(r_movie.text, 'html.parser')
        movie = get_movie_info(soup_movie)
        all_movies_info.append(movie)
        #print(movie['Name'])
    scraped_df = pd.DataFrame(all_movies_info)
    scraped_df.to_csv('scraped_data'+filename+".csv", index=False)
    scraped_df.head()

got all links


  5%|▌         | 5/100 [00:55<17:33, 11.09s/it]


KeyboardInterrupt: 

In [ ]:
finance_table = soup_movie.find('table', id='movie_finances')
information = finance_table.find_all('td')

for i in range(len(information)):
    info = information[i]
    #print(info)
    if not info.find('a') and not info.get("colspan"):
        if info.get('class') and info.get('class')[0] == 'data':
            if information[i-1].find('b').text == 'Domestic Box Office':
                current_movie['domestic_gross'] = info.text
            if information[i-1].find('b').text == 'International Box Office':
                current_movie['foreign_gross'] = info.text
            if information[i-1].find('b').text == 'Worldwide Box Office':
                current_movie['worldwide_gross'] = info.text
